In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import pickle
import torch
from tqdm import tqdm, trange
# import faiss
import gc
import time
from ast import literal_eval
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MultiLabelBinarizer
from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import classification_report

In [2]:
!pip install imblearn
!pip install pyarrow
!pip install fastparquet
!pip install wandb -qU

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 258.3/258.3 kB 12.6 MB/s eta 0:00:00

[notice] A new release of pip is available: 23.0.1 -> 24.2
[notice] To update, run: pip install --upgrade pip
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 40.0/40.0 MB 23.0 MB/s eta 0:00:0000:0100:01

[notice] A new release of pip is available: 23.0.1 -> 24.2
[notice] To update, run: pip install --upgrade pip
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.7/1.7 MB 37.7 MB/s eta 0:00:0000:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.0/2.0 MB 61.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 177.6/177.6 kB 16.0 MB/s eta 0:00:00

[notice] A new release of pip is available: 23.0.1 -> 24.2
[notice] To update, run: pip install --upgrade pip
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
elegy 0.8.6 requires wandb<0.13.0,>=0.12.10, but you have wandb 0.17.5 which

In [3]:
from imblearn.under_sampling import RandomUnderSampler
import wandb

In [4]:
wandb.require("core")
wandb.login()

wandb: Using wandb-core as the SDK backend. Please refer to https://wandb.me/wandb-core for more information.
wandb: Logging into wandb.ai. (Learn how to deploy a W&B server locally: https://wandb.me/wandb-server)
wandb: You can find your API key in your browser here: https://wandb.ai/authorize
wandb: Paste an API key from your profile and hit enter, or press ctrl+c to quit:

  ········································


wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc


True

In [ ]:
wandb.init(
    project="AIST-CodeContest", job_type='metric_calculation')

In [6]:
import torch
import torch_xla
import torch_xla.core.xla_model as xm

t = torch.randn(2, 2, device=xm.xla_device())
print(t.device)
print(t)

xla:0
tensor([[-0.6989, -0.0987],
        [ 0.7337, -0.9071]], device='xla:0')


E0729 10:16:08.084927550     486 oauth2_credentials.cc:236]            oauth_fetch: UNKNOWN:C-ares status is not ARES_SUCCESS qtype=A name=metadata.google.internal. is_balancer=0: Domain name not found {grpc_status:2, created_time:"2024-07-29T10:16:08.084911455+00:00"}


In [7]:
print("ok")

ok


# Prepare Data

In [8]:
submissions_df = pd.read_parquet(
    '/kaggle/input/code-contests-with-embeddings/code_contests_cf_filtered_exploded_truncated_with_embs.parquet',
    use_threads = False
)

#submissions_df['problem_url'] =  submissions_df['problem_url'].astype("category")
#submissions_df['problem_id'] = submissions_df.problem_url.cat.codes
submissions_df

,submission_id,name,description,cf_contest_id,cf_index,cf_points,cf_rating,cf_tags,source_code,problem_url,codebert_cpp_embeddings,codebert_embeddings
0,397026,1_A. Theatre Square,Theatre Square in the capital city of Berland ...,1,A,<NA>,1000,[math],#include <bits/stdc++.h>\nint main() {\n unsi...,/contest/1/problem/A,"[0.22658451, 0.32135278, -0.019483238, -0.3847...","[-0.28651828, -0.019488286, 0.2861307, 0.19298..."
1,397027,1_A. Theatre Square,Theatre Square in the capital city of Berland ...,1,A,<NA>,1000,[math],#include <bits/stdc++.h>\nint main() {\n long...,/contest/1/problem/A,"[0.301358, 0.4700294, -0.039679203, -0.3457937...","[-0.28672487, -0.010818661, 0.23493826, 0.2357..."
2,397028,1_A. Theatre Square,Theatre Square in the capital city of Berland ...,1,A,<NA>,1000,[math],#include <bits/stdc++.h>\nusing namespace std;...,/contest/1/problem/A,"[0.24211046, 0.5052995, -0.04573048, -0.360875...","[-0.26832008, -0.033796232, 0.2554274, 0.20098..."
3,397029,1_A. Theatre Square,Theatre Square in the capital city of Berland ...,1,A,<NA>,1000,[math],#include <bits/stdc++.h>\nusing namespace std;...,/contest/1/problem/A,"[0.25702566, 0.30060184, 0.09671167, -0.173451...","[-0.2108263, -0.11545009, 0.21462078, 0.153441..."
4,397030,1_A. Theatre Square,Theatre Square in the capital city of Berland ...,1,A,<NA>,1000,[math],#include <bits/stdc++.h>\nusing namespace std;...,/contest/1/problem/A,"[0.2818388, 0.5189903, -0.03950362, -0.3319832...","[-0.30226648, 0.010255673, 0.22609307, 0.18772..."
...,...,...,...,...,...,...,...,...,...,...,...,...
1333422,105493,999_F. Cards and Joy,There are n players sitting at the card table....,999,F,<NA>,2000,[dp],#include <bits/stdc++.h>\nusing namespace std;...,/contest/999/problem/F,"[0.2214258, 0.34111664, -0.0005975967, -0.3527...","[-0.34487352, -0.043685403, 0.23232168, 0.1735..."
1333423,105494,999_F. Cards and Joy,There are n players sitting at the card table....,999,F,<NA>,2000,[dp],#include <bits/stdc++.h>\nusing namespace std;...,/contest/999/problem/F,"[0.10929607, 0.29541174, -0.03201199, -0.37105...","[-0.28139487, -0.09595364, 0.16795102, 0.3584,..."
1333424,105495,999_F. Cards and Joy,There are n players sitting at the card table....,999,F,<NA>,2000,[dp],#include <bits/stdc++.h>\n#pragma GCC optimize...,/contest/999/problem/F,"[0.2277699, 0.33140126, -0.018488321, -0.25750...","[-0.30307686, -0.041024357, 0.2352449, 0.30709..."
1333425,105496,999_F. Cards and Joy,There are n players sitting at the card table....,999,F,<NA>,2000,[dp],#include <bits/stdc++.h>\nusing namespace std;...,/contest/999/problem/F,"[0.26763135, 0.2902782, -0.010922076, -0.30965...","[-0.31702673, -0.043351483, 0.2525895, 0.21798..."


In [9]:
with open('/kaggle/input/aist-train-model/embeddings_MLP.pkl', 'rb') as f:
    mlp_embeddings = pickle.load(f)
mlp_embeddings = torch.stack(mlp_embeddings).squeeze().numpy()
len(mlp_embeddings)

1333427

In [10]:
submissions_df['mlp_embeddings'] = list(mlp_embeddings)
submissions_df.head()

,submission_id,name,description,cf_contest_id,cf_index,cf_points,cf_rating,cf_tags,source_code,problem_url,codebert_cpp_embeddings,codebert_embeddings,mlp_embeddings
0,397026,1_A. Theatre Square,Theatre Square in the capital city of Berland ...,1,A,<NA>,1000,[math],#include <bits/stdc++.h>\nint main() {\n unsi...,/contest/1/problem/A,"[0.22658451, 0.32135278, -0.019483238, -0.3847...","[-0.28651828, -0.019488286, 0.2861307, 0.19298...","[1.0991068, 0.035263382, 3.264473, -5.07334, -..."
1,397027,1_A. Theatre Square,Theatre Square in the capital city of Berland ...,1,A,<NA>,1000,[math],#include <bits/stdc++.h>\nint main() {\n long...,/contest/1/problem/A,"[0.301358, 0.4700294, -0.039679203, -0.3457937...","[-0.28672487, -0.010818661, 0.23493826, 0.2357...","[-2.2484708, 3.5446873, 3.427662, -4.1564198, ..."
2,397028,1_A. Theatre Square,Theatre Square in the capital city of Berland ...,1,A,<NA>,1000,[math],#include <bits/stdc++.h>\nusing namespace std;...,/contest/1/problem/A,"[0.24211046, 0.5052995, -0.04573048, -0.360875...","[-0.26832008, -0.033796232, 0.2554274, 0.20098...","[1.8236251, 5.3814106, 3.6940706, 0.15576231, ..."
3,397029,1_A. Theatre Square,Theatre Square in the capital city of Berland ...,1,A,<NA>,1000,[math],#include <bits/stdc++.h>\nusing namespace std;...,/contest/1/problem/A,"[0.25702566, 0.30060184, 0.09671167, -0.173451...","[-0.2108263, -0.11545009, 0.21462078, 0.153441...","[5.284894, 0.6567197, 0.24589896, -7.1280603, ..."
4,397030,1_A. Theatre Square,Theatre Square in the capital city of Berland ...,1,A,<NA>,1000,[math],#include <bits/stdc++.h>\nusing namespace std;...,/contest/1/problem/A,"[0.2818388, 0.5189903, -0.03950362, -0.3319832...","[-0.30226648, 0.010255673, 0.22609307, 0.18772...","[0.37458837, 0.2923608, 4.48598, -4.9987383, -..."


## make tag id

In [11]:
submissions_df = submissions_df.dropna(subset=['cf_tags'])

In [12]:
len(submissions_df)

1322512

## Multilabel Binarizer

In [13]:
mlb = MultiLabelBinarizer()
y_classes = mlb.fit_transform(submissions_df.cf_tags)

In [14]:
a, b = train_test_split(submissions_df.problem_url.value_counts(), random_state=42)

In [15]:
bert_cpp_train = submissions_df[submissions_df.problem_url.isin(a.index)].codebert_cpp_embeddings
bert_train = submissions_df[submissions_df.problem_url.isin(a.index)].codebert_embeddings
mlp_train = submissions_df[submissions_df.problem_url.isin(a.index)].mlp_embeddings

bert_cpp_test = submissions_df[submissions_df.problem_url.isin(b.index)].codebert_cpp_embeddings
bert_test = submissions_df[submissions_df.problem_url.isin(b.index)].codebert_embeddings
mlp_test = submissions_df[submissions_df.problem_url.isin(b.index)].mlp_embeddings

# KNN

## Test on 'string' tag

In [16]:
import os

cpu_c = os.cpu_count()

In [23]:
train_str = y_classes[submissions_df.problem_url.isin(a.index)][:, 33] # mlb.classes_[33] == strings
test_str = y_classes[submissions_df.problem_url.isin(b.index)][:, 33]

In [24]:
rus = RandomUnderSampler(random_state=42)

bert_train_str, y_train_str = rus.fit_resample(np.stack(bert_train), train_str)
bert_test_str, y_test_str = rus.fit_resample(np.stack(bert_test), test_str)

mlp_train_str, y_train_str = rus.fit_resample(np.stack(mlp_train), train_str)
mlp_test_str, y_test_str = rus.fit_resample(np.stack(mlp_test), test_str)

In [25]:
knn = KNeighborsClassifier(n_jobs=cpu_c, algorithm='brute', metric='cosine')

knn.fit(mlp_train_str, y_train_str)

KNeighborsClassifier(algorithm='brute', metric='cosine', n_jobs=96)

In [26]:
%%time
brute_cosine = knn.predict(mlp_test_str)

CPU times: user 18min 29s, sys: 15min 23s, total: 33min 52s
Wall time: 4min 21s


In [27]:
print(classification_report(y_test_str, brute_cosine))

              precision    recall  f1-score   support

           0       0.83      0.87      0.85     28495
           1       0.87      0.83      0.85     28495

    accuracy                           0.85     56990
   macro avg       0.85      0.85      0.85     56990
weighted avg       0.85      0.85      0.85     56990



## KNN cycle

In [17]:
def perform_knn_classification(X_train, X_test, y_train, y_test, n_jobs=90):
    rus = RandomUnderSampler(random_state=42)
    X_train_us, y_train_us = rus.fit_resample(np.stack(X_train), y_train)
    X_test_us, y_test_us = rus.fit_resample(np.stack(X_test), y_test)
    knn = KNeighborsClassifier(n_jobs=n_jobs, metric='cosine', algorithm='brute')
    knn.fit(X_train_us, y_train_us)
    y_pred_us = knn.predict(X_test_us)
    return (y_test_us, y_pred_us)

In [18]:
knn_results_mlp = {}

In [20]:
mlb.classes_[20:]

array(['greedy', 'hashing', 'implementation', 'interactive', 'math',
       'matrices', 'meet-in-the-middle', 'number theory', 'probabilities',
       'schedules', 'shortest paths', 'sortings',
       'string suffix structures', 'strings', 'ternary search', 'trees',
       'two pointers'], dtype=object)

In [21]:
for ind, label in tqdm(enumerate(mlb.classes_[20:]), total=len(mlb.classes_[20:])): 
    knn_results_mlp[label] = perform_knn_classification(mlp_train, mlp_test,
                                                           y_classes[submissions_df.problem_url.isin(a.index)][:, ind],
                                                           y_classes[submissions_df.problem_url.isin(b.index)][:, ind],
                                                       n_jobs=cpu_c)
    with open("knn_mlp_classification.pkl", 'wb') as f:
        pickle.dump(knn_results_mlp, f)

    #wandb save
    artifact = wandb.Artifact('knn_mlp_classification_part2', type='metrics')
    artifact.add_file('knn_mlp_classification.pkl')
    wandb.log_artifact(artifact)

100%|██████████| 17/17 [1:15:37<00:00, 266.92s/it]


In [22]:
for label, res in knn_results_mlp.items():
    print(label)
    print(classification_report(res[0], res[1]))

greedy
              precision    recall  f1-score   support

           0       0.65      0.80      0.71      5410
           1       0.74      0.56      0.64      5410

    accuracy                           0.68     10820
   macro avg       0.69      0.68      0.68     10820
weighted avg       0.69      0.68      0.68     10820

hashing
              precision    recall  f1-score   support

           0       0.64      0.86      0.73        84
           1       0.79      0.52      0.63        84

    accuracy                           0.69       168
   macro avg       0.71      0.69      0.68       168
weighted avg       0.71      0.69      0.68       168

implementation
              precision    recall  f1-score   support

           0       0.74      0.79      0.77     32499
           1       0.78      0.72      0.75     32499

    accuracy                           0.76     64998
   macro avg       0.76      0.76      0.76     64998
weighted avg       0.76      0.76      0.76 

In [ ]:
with open("knn_mlp_classification.pkl", 'wb') as f:
    pickle.dump(knn_results_mlp, f)

#wandb save
artifact = wandb.Artifact('knn_mlp_classification_part2', type='metrics')
artifact.add_file('knn_mlp_classification.pkl')
wandb.log_artifact(artifact)
wandb.finish()